In [49]:
import pandas as pd
import implicit
import scipy.sparse as sparse
import random
import numpy as np
from tqdm.notebook import tqdm

In [50]:
!export MKL_NUM_THREADS=1

In [51]:
rnames = list(str.split('UserID::MovieID::Rating::Timestamp','::'))
ratings=pd.read_table('../data/ratings.dat',sep='::',header=None, names=rnames, engine='python')

In [52]:
unames = list(str.split('UserID::Gender::Age::Occupation::Zip-code',"::"))
users = pd.read_table('../data/users.dat', sep='::', header=None, names=unames)

mnames = list(str.split('MovieID::Title::Genres',"::"))
movies = pd.read_csv('../data/movies.dat',sep='::',encoding='ISO-8859-1',header=None,names=["MovieID", "Title", "Genres"])

/Users/daniilzhilin/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


<h4> Делить на train test нужно по какому то временному промежутку, 
    плюс к этому мы хотим провести оптимизацию по recall@100, 
    значит нам нужно чтобы в тесте были пользователи с более менее нормальным количеством проставленных оценок </h4>

In [7]:
ratings.dtypes

UserID       int64
MovieID      int64
Rating       int64
Timestamp    int64
dtype: object

In [8]:
ratings['date'] = pd.to_datetime(ratings.Timestamp, unit='s')

In [9]:
ratings.date.min()

Timestamp('2000-04-25 23:05:32')

In [10]:
ratings.date.max()

Timestamp('2003-02-28 17:49:50')

In [27]:
ratings_2003_mask = ratings.date > '2003-01-01'

In [28]:
ratings[ratings_2003_mask].UserID.nunique()

178

In [21]:
ratings[ratings_2003_mask].groupby('UserID').Rating.transform('count').mean()

115.08303464755078

В среднем у пользователей за 2 месяца 115 выставленных оценок, думаю можно на тест это оставить

In [29]:
train = ratings[ratings.date < '2003-01-01']

In [30]:
test = ratings[ratings_2003_mask]

In [31]:
train

,UserID,MovieID,Rating,Timestamp,date
0,1,1193,5,978300760,2000-12-31 22:12:40
1,1,661,3,978302109,2000-12-31 22:35:09
2,1,914,3,978301968,2000-12-31 22:32:48
3,1,3408,4,978300275,2000-12-31 22:04:35
4,1,2355,5,978824291,2001-01-06 23:38:11
...,...,...,...,...,...
1000204,6040,1091,1,956716541,2000-04-26 02:35:41
1000205,6040,1094,5,956704887,2000-04-25 23:21:27
1000206,6040,562,5,956704746,2000-04-25 23:19:06
1000207,6040,1096,4,956715648,2000-04-26 02:20:48


In [32]:
train.drop(['Timestamp'], axis=1, inplace=True)

/Users/daniilzhilin/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [40]:
sparse_item_user_train = sparse.csr_matrix((train['Rating'].astype(float), (train['MovieID'], train['UserID'])))
sparse_user_item_train = sparse.csr_matrix((train['Rating'].astype(float), (train['UserID'], train['MovieID'])))

In [43]:
model = implicit.als.AlternatingLeastSquares(factors=50)

In [44]:
model.fit(sparse_item_user_train)

  0%|          | 0/15 [00:00<?, ?it/s]

In [48]:
userid = 100
model.recommend(userid, sparse_user_item[userid], N=100)

(array([3618, 4277, 4647, 4169,  524, 1448, 3391, 1680, 1150,  710, 2857,
        3067, 3841,  692, 3425,  331, 1069,  752, 3539,  352, 3507,  549,
        5536, 1181, 2909, 3526, 1010, 4543, 2304, 3224, 3808, 4578, 5795,
        4682, 1284, 2453, 4116,  319, 2376, 1015,  660, 3829, 4312,  881,
        3650, 4482, 2063, 4508, 1285, 3483, 1449, 1203, 2793, 3778,   53,
        4033, 3285, 5643, 1835,  721, 5039, 4626, 5493, 5682, 4404, 4139,
        2986, 3705, 3471, 5675,  839,  999, 5433, 2743, 4344, 2077, 4088,
        2230, 1889, 3336, 2820, 1980,  149,  699, 4448, 2872,  482, 5614,
        2116, 5046, 1194,  935, 2098, 4425, 5837, 2825, 2777, 1613,  970,
        3491], dtype=int32),
 array([1.0434023 , 0.9347564 , 0.92471904, 0.9169564 , 0.87786937,
        0.83858675, 0.83822757, 0.83707523, 0.82841325, 0.8051653 ,
        0.7855506 , 0.77639025, 0.7755493 , 0.7669691 , 0.7588082 ,
        0.75657505, 0.7491331 , 0.73797786, 0.73513395, 0.73502475,
        0.7348834 , 0.7174138 , 0